![DSL_logo](dsl_logo.png)


# Advanced Text Analysis with Python

Welcome to the Digital Scholarship Lab introduction to Text Analysis with Python class. In this class we'll learn the basics of text analysis:

- Topic Modeling
- Text Similarity


This workshop assumes you've completed our Introduction to Text Analysis [workshop](https://brockdsl.github.io/Text_Analysis_with_Python/)

It would be useful to consult our Introduction to Python 


We'll use the Zoom's chat feature to interact.

Be sure to enable line numbers by looking for the 'gear' icon and checking the box in the 'Editor' panel.